In [2]:
import pandas as pd

In [3]:
channel1 = 'Scikitlearn'
channel2 = 'Angular'

## Respondent Analysis

### Scikitlearn

In [21]:
df_scikit_profile = pd.read_csv(channel1+'/'+channel1+'_profile.csv')

In [22]:
df_scikit_profile.head()

,user_Id,cluster_Id,answer_date,answers
0,54d4a1d6db8155e6700f853b,12,2015-02-06 11:14:00,I'm with family this week and so not super act...
1,54d4a1d6db8155e6700f853b,61,2015-02-24 22:46:00,cool\r\nok\r\n
2,54d4a1d6db8155e6700f853b,69,2015-02-25 18:59:00,yeah that sounds like a good idea\r\n
3,54d4a1d6db8155e6700f853b,72,2015-02-25 19:14:00,you mean == 1\r\n
4,54d4a1d6db8155e6700f853b,75,2015-02-25 19:19:00,that is also weird ;)\r\n


In [23]:
df_scikit_sorted = df_scikit_profile.groupby(df_scikit_profile.user_Id)["user_Id"].count().reset_index(name='count').sort_values(['count'], ascending=False,ignore_index= True)

In [24]:
df_scikit_sorted.describe()

,count
count,106.000000
mean,5.773585
std,19.789503
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,186.000000


In [25]:
len(df_scikit_sorted[df_scikit_sorted['count'] >= 10])

9

In [26]:
len(df_scikit_sorted[df_scikit_sorted['count'] >= 100])

1

In [28]:
print("dialogues with response:",len(set(df_scikit_profile["cluster_Id"])))

dialogues with response: 595


In [52]:
df_scikit_issue = pd.read_json(channel1+'/'+channel1+'_issue.json')

In [53]:
df_scikit_issue.head()

,Id,asker,time,content,lines
0,1,541a528b163965c9bc2053de,2014-09-18 03:36,Hi all!\n,1
1,2,544906e2db8155e6700cdd16,2014-10-23 19:59,I don't know if this is the right place but I ...,2
2,3,544906e2db8155e6700cdd16,2014-10-23 20:01,My first approach was to convert each feature ...,2
3,4,53a5cf04a9176b500d1ced1a,2015-01-23 15:38,This message is without content. I wonder if a...,1
4,5,541a528b163965c9bc2053de,2015-01-23 16:49,"@mac2bua, I just saw your message. It's probab...",1


### Angular

In [77]:
df_angular_profile = pd.read_csv(channel2+'/'+channel2+'_profile.csv')

In [78]:
df_angular_profile.head()

,user_Id,cluster_Id,answer_date,answers
0,54e6ee3b15522ed4b3dc3c24,15,2015-03-08 18:01:00,currently only working in chrome until shadow ...
1,54e6ee3b15522ed4b3dc3c24,20,2015-03-09 02:39:00,::shadow\nhttp://www.html5rocks.com/en/tutoria...
2,54e6ee3b15522ed4b3dc3c24,100,2015-03-13 19:48:00,@mhevery that was my worry too (Dart version ...
3,54e6ee3b15522ed4b3dc3c24,113,2015-03-14 02:46:00,you donat need to inject it since itas going t...
4,54e6ee3b15522ed4b3dc3c24,118,2015-03-14 07:23:00,what version of node?\n


In [13]:
df_angular_sorted = df_angular_profile.groupby(df_angular_profile.user_Id)["user_Id"].count().reset_index(name='count').sort_values(['count'], ascending=False,ignore_index= True)

In [14]:
df_angular_sorted.describe()

,count
count,6707.000000
mean,21.891904
std,232.142550
min,1.000000
25%,1.000000
50%,3.000000
75%,7.000000
max,11700.000000


In [15]:
len(df_angular_sorted[df_angular_sorted['count'] >= 10])

1317

In [16]:
len(df_angular_sorted[df_angular_sorted['count'] >= 100])

163

In [17]:
print("dialogues with response:",len(set(df_angular_profile["cluster_Id"])))

dialogues with response: 122791


In [79]:
df_angular_issue = pd.read_json(channel2+'/'+channel2+'_issue.json')

In [80]:
df_angular_issue.head()

,Id,asker,time,content,lines
0,1,54c89b1cdb8155e6700f2ac3,2015-03-07 21:41,Awesome!\n,1
1,2,54e6ee3b15522ed4b3dc3c24,2015-03-07 21:53,"sweet, now we can ask angular2 questions in re...",1
2,3,530cd1eb5e986b0712efb4d2,2015-03-07 22:04,Yes!\n,1
3,4,54e6ee3b15522ed4b3dc3c24,2015-03-07 22:06,we're missing the [View Source] [Improve this ...,1
4,5,530cd1eb5e986b0712efb4d2,2015-03-07 23:18,Good idea can you file an issue with repo?\n,1


## Tags Matching

In [4]:
import nltk
from nltk.corpus import stopwords
import requests
import json
import os.path
from requests.exceptions import ConnectionError
import time
import sys

In [5]:
with open('st_token.txt', 'r') as fp:
    st_token = fp.readline()

In [91]:
def tags_match(df,tags, tags_stem):
    profile_tags = []
    total_tags = set()
    for answer in df:
        #print(answer)
        tags_matched = []
        answer = str(answer)
        if len(str(answer)) == 0:
            profile_tags.append(tags_matched)
            continue
        
        for tag, tag_stem in zip(tags,tags_stem):
            
            '''
            if str(tag) in answer:
                tags_matched.append(tag)
                total_tags.add(tag)
            '''
                
            
            is_match = False
            for token in tag_stem:
                
                start_index = answer.find(str(token))
                if start_index != -1:
                    #print(token)
                    if ((start_index == 0) or (not answer[start_index-1].isalpha())) and \
                        ((start_index+len(str(token)) == len(answer)) or (not answer[start_index+len(str(token))].isalpha())):
                        is_match = True
                    else:
                        is_match = False
                        break
                else:
                    is_match = False
                    break
            
            if is_match:
                tags_matched.append(tag)
                total_tags.add(tag)
    
        profile_tags.append(tags_matched)
        
    print("total tags matched:",len(total_tags))
    
    return profile_tags,total_tags

In [7]:
class PorterStemmer:

    def __init__(self):
        """The main part of the stemming algorithm starts here.
        b is a buffer holding a word to be stemmed. The letters are in b[k0],
        b[k0+1] ... ending at b[k]. In fact k0 = 0 in this demo program. k is
        readjusted downwards as the stemming progresses. Zero termination is
        not in fact used in the algorithm.

        Note that only lower case sequences are stemmed. Forcing to lower case
        should be done before stem(...) is called.
        """

        self.b = ""  # buffer for word to be stemmed
        self.k = 0
        self.k0 = 0
        self.j = 0   # j is a general offset into the string

    def cons(self, i):
        """cons(i) is TRUE <=> b[i] is a consonant."""
        if self.b[i] == 'a' or self.b[i] == 'e' or self.b[i] == 'i' or self.b[i] == 'o' or self.b[i] == 'u':
            return 0
        if self.b[i] == 'y':
            if i == self.k0:
                return 1
            else:
                return (not self.cons(i - 1))
        return 1

    def m(self):
        """m() measures the number of consonant sequences between k0 and j.
        if c is a consonant sequence and v a vowel sequence, and <..>
        indicates arbitrary presence,

           <c><v>       gives 0
           <c>vc<v>     gives 1
           <c>vcvc<v>   gives 2
           <c>vcvcvc<v> gives 3
           ....
        """
        n = 0
        i = self.k0
        while 1:
            if i > self.j:
                return n
            if not self.cons(i):
                break
            i = i + 1
        i = i + 1
        while 1:
            while 1:
                if i > self.j:
                    return n
                if self.cons(i):
                    break
                i = i + 1
            i = i + 1
            n = n + 1
            while 1:
                if i > self.j:
                    return n
                if not self.cons(i):
                    break
                i = i + 1
            i = i + 1

    def vowelinstem(self):
        """vowelinstem() is TRUE <=> k0,...j contains a vowel"""
        for i in range(self.k0, self.j + 1):
            if not self.cons(i):
                return 1
        return 0

    def doublec(self, j):
        """doublec(j) is TRUE <=> j,(j-1) contain a double consonant."""
        if j < (self.k0 + 1):
            return 0
        if (self.b[j] != self.b[j-1]):
            return 0
        return self.cons(j)

    def cvc(self, i):
        """cvc(i) is TRUE <=> i-2,i-1,i has the form consonant - vowel - consonant
        and also if the second c is not w,x or y. this is used when trying to
        restore an e at the end of a short  e.g.

           cav(e), lov(e), hop(e), crim(e), but
           snow, box, tray.
        """
        if i < (self.k0 + 2) or not self.cons(i) or self.cons(i-1) or not self.cons(i-2):
            return 0
        ch = self.b[i]
        if ch == 'w' or ch == 'x' or ch == 'y':
            return 0
        return 1

    def ends(self, s):
        """ends(s) is TRUE <=> k0,...k ends with the string s."""
        length = len(s)
        if s[length - 1] != self.b[self.k]: # tiny speed-up
            return 0
        if length > (self.k - self.k0 + 1):
            return 0
        if self.b[self.k-length+1:self.k+1] != s:
            return 0
        self.j = self.k - length
        return 1

    def setto(self, s):
        """setto(s) sets (j+1),...k to the characters in the string s, readjusting k."""
        length = len(s)
        self.b = self.b[:self.j+1] + s + self.b[self.j+length+1:]
        self.k = self.j + length

    def r(self, s):
        """r(s) is used further down."""
        if self.m() > 0:
            self.setto(s)

    def step1ab(self):
        """step1ab() gets rid of plurals and -ed or -ing. e.g.

           caresses  ->  caress
           ponies    ->  poni
           ties      ->  ti
           caress    ->  caress
           cats      ->  cat

           feed      ->  feed
           agreed    ->  agree
           disabled  ->  disable

           matting   ->  mat
           mating    ->  mate
           meeting   ->  meet
           milling   ->  mill
           messing   ->  mess

           meetings  ->  meet
        """
        if self.b[self.k] == 's':
            if self.ends("sses"):
                self.k = self.k - 2
            elif self.ends("ies"):
                self.setto("i")
            elif self.b[self.k - 1] != 's':
                self.k = self.k - 1
        if self.ends("eed"):
            if self.m() > 0:
                self.k = self.k - 1
        elif (self.ends("ed") or self.ends("ing")) and self.vowelinstem():
            self.k = self.j
            if self.ends("at"):   self.setto("ate")
            elif self.ends("bl"): self.setto("ble")
            elif self.ends("iz"): self.setto("ize")
            elif self.doublec(self.k):
                self.k = self.k - 1
                ch = self.b[self.k]
                if ch == 'l' or ch == 's' or ch == 'z':
                    self.k = self.k + 1
            elif (self.m() == 1 and self.cvc(self.k)):
                self.setto("e")

    def step1c(self):
        """step1c() turns terminal y to i when there is another vowel in the stem."""
        if (self.ends("y") and self.vowelinstem()):
            self.b = self.b[:self.k] + 'i' + self.b[self.k+1:]

    def step2(self):
        """step2() maps double suffices to single ones.
        so -ization ( = -ize plus -ation) maps to -ize etc. note that the
        string before the suffix must give m() > 0.
        """
        if self.b[self.k - 1] == 'a':
            if self.ends("ational"):   self.r("ate")
            elif self.ends("tional"):  self.r("tion")
        elif self.b[self.k - 1] == 'c':
            if self.ends("enci"):      self.r("ence")
            elif self.ends("anci"):    self.r("ance")
        elif self.b[self.k - 1] == 'e':
            if self.ends("izer"):      self.r("ize")
        elif self.b[self.k - 1] == 'l':
            if self.ends("bli"):       self.r("ble") # --DEPARTURE--
            # To match the published algorithm, replace this phrase with
            #   if self.ends("abli"):      self.r("able")
            elif self.ends("alli"):    self.r("al")
            elif self.ends("entli"):   self.r("ent")
            elif self.ends("eli"):     self.r("e")
            elif self.ends("ousli"):   self.r("ous")
        elif self.b[self.k - 1] == 'o':
            if self.ends("ization"):   self.r("ize")
            elif self.ends("ation"):   self.r("ate")
            elif self.ends("ator"):    self.r("ate")
        elif self.b[self.k - 1] == 's':
            if self.ends("alism"):     self.r("al")
            elif self.ends("iveness"): self.r("ive")
            elif self.ends("fulness"): self.r("ful")
            elif self.ends("ousness"): self.r("ous")
        elif self.b[self.k - 1] == 't':
            if self.ends("aliti"):     self.r("al")
            elif self.ends("iviti"):   self.r("ive")
            elif self.ends("biliti"):  self.r("ble")
        elif self.b[self.k - 1] == 'g': # --DEPARTURE--
            if self.ends("logi"):      self.r("log")
        # To match the published algorithm, delete this phrase

    def step3(self):
        """step3() dels with -ic-, -full, -ness etc. similar strategy to step2."""
        if self.b[self.k] == 'e':
            if self.ends("icate"):     self.r("ic")
            elif self.ends("ative"):   self.r("")
            elif self.ends("alize"):   self.r("al")
        elif self.b[self.k] == 'i':
            if self.ends("iciti"):     self.r("ic")
        elif self.b[self.k] == 'l':
            if self.ends("ical"):      self.r("ic")
            elif self.ends("ful"):     self.r("")
        elif self.b[self.k] == 's':
            if self.ends("ness"):      self.r("")

    def step4(self):
        """step4() takes off -ant, -ence etc., in context <c>vcvc<v>."""
        if self.b[self.k - 1] == 'a':
            if self.ends("al"): pass
            else: return
        elif self.b[self.k - 1] == 'c':
            if self.ends("ance"): pass
            elif self.ends("ence"): pass
            else: return
        elif self.b[self.k - 1] == 'e':
            if self.ends("er"): pass
            else: return
        elif self.b[self.k - 1] == 'i':
            if self.ends("ic"): pass
            else: return
        elif self.b[self.k - 1] == 'l':
            if self.ends("able"): pass
            elif self.ends("ible"): pass
            else: return
        elif self.b[self.k - 1] == 'n':
            if self.ends("ant"): pass
            elif self.ends("ement"): pass
            elif self.ends("ment"): pass
            elif self.ends("ent"): pass
            else: return
        elif self.b[self.k - 1] == 'o':
            if self.ends("ion") and (self.b[self.j] == 's' or self.b[self.j] == 't'): pass
            elif self.ends("ou"): pass
            # takes care of -ous
            else: return
        elif self.b[self.k - 1] == 's':
            if self.ends("ism"): pass
            else: return
        elif self.b[self.k - 1] == 't':
            if self.ends("ate"): pass
            elif self.ends("iti"): pass
            else: return
        elif self.b[self.k - 1] == 'u':
            if self.ends("ous"): pass
            else: return
        elif self.b[self.k - 1] == 'v':
            if self.ends("ive"): pass
            else: return
        elif self.b[self.k - 1] == 'z':
            if self.ends("ize"): pass
            else: return
        else:
            return
        if self.m() > 1:
            self.k = self.j

    def step5(self):
        """step5() removes a final -e if m() > 1, and changes -ll to -l if
        m() > 1.
        """
        self.j = self.k
        if self.b[self.k] == 'e':
            a = self.m()
            if a > 1 or (a == 1 and not self.cvc(self.k-1)):
                self.k = self.k - 1
        if self.b[self.k] == 'l' and self.doublec(self.k) and self.m() > 1:
            self.k = self.k -1

    def stem(self, p, i, j):
        """In stem(p,i,j), p is a char pointer, and the string to be stemmed
        is from p[i] to p[j] inclusive. Typically i is zero and j is the
        offset to the last character of a string, (p[j+1] == '\0'). The
        stemmer adjusts the characters p[i] ... p[j] and returns the new
        end-point of the string, k. Stemming never increases word length, so
        i <= k <= j. To turn the stemmer into a module, declare 'stem' as
        extern, and delete the remainder of this file.
        """
        # copy the parameters into statics
        self.b = p
        self.k = j
        self.k0 = i
        if self.k <= self.k0 + 1:
            return self.b # --DEPARTURE--

        # With this line, strings of length 1 or 2 don't go through the
        # stemming process, although no mention is made of this in the
        # published algorithm. Remove the line to match the published
        # algorithm.

        self.step1ab()
        self.step1c()
        self.step2()
        self.step3()
        self.step4()
        self.step5()
        return self.b[self.k0:self.k+1]

In [74]:
df_tags = pd.read_csv('tags.csv')

In [75]:
len(df_tags)

63581

In [76]:
df_tags.head()

,Id,TagName,Count,ExcerptPostId,WikiPostId,IsModeratorOnly,IsRequired
0,1,.net,322128,3624959.0,3607476.0,NaN,NaN
1,2,html,1141471,3673183.0,3673182.0,NaN,NaN
2,3,javascript,2413015,3624960.0,3607052.0,NaN,NaN
3,4,css,768321,3644670.0,3644669.0,NaN,NaN
4,5,php,1443156,3624936.0,3607050.0,NaN,NaN


In [77]:
df_tags = df_tags.sort_values(['Count'], ascending=False,ignore_index= True)

In [78]:
df_tags = df_tags[df_tags["Count"] >= 1000]

In [79]:
df_tags.head()

,Id,TagName,Count,ExcerptPostId,WikiPostId,IsModeratorOnly,IsRequired
0,3,javascript,2413015,3624960.0,3607052.0,NaN,NaN
1,16,python,2006826,3624965.0,3607014.0,NaN,NaN
2,17,java,1860638,3624966.0,3607018.0,NaN,NaN
3,9,c#,1553691,3624962.0,3607007.0,NaN,NaN
4,5,php,1443156,3624936.0,3607050.0,NaN,NaN


In [80]:
tags_clean = set()
for tag in df_tags["TagName"]:
    if len(str(tag)) > 1 and str(tag) not in stopwords.words('english'):
        tags_clean.add(str(tag))

In [81]:
len(tags_clean)

5257

In [82]:
tags_desc = []
visited = set()

In [65]:
#retreive tag description
for tag in tags_clean:
    if tag in visited:
        continue
    
    print(tag)
    tag_link = "https://api.stackexchange.com/2.3/tags/"+tag+"/wikis?site=stackoverflow"+"&key="+st_token
    #api_link = requests.get(tag_link)
    #tag_data = api_link.json()
    flag = True
    tag_data = None
    
    while flag:
        try:
            api_link = requests.get(tag_link)          
            tag_data = api_link.json()
            flag = False
        except:
            print("Connect Error!")
            print("Sleep 50 seconds!")
            time.sleep(50)
    
    if len(tag_data) == 0 or api_link.status_code == 404 or "error_id" in tag_data.keys():
        tags_desc.append("")
        continue         
    
    if len(tag_data["items"]) > 0 and "excerpt" in tag_data["items"][0].keys():
        #print(tag_data["items"][0]["excerpt"])
        tags_desc.append(tag_data["items"][0]["excerpt"])
    else:
        tags_desc.append("")
        
    visited.add(tag)
    

facebook-c#-sdk
firebase-hosting
powerquery
calculator
blender
selenium-rc
solidity
excel-formula
sails.js
survey
data-access-layer
uiactionsheet
centos6
core-data
yii
heroku
quickbooks
picker
svg
preview
outliers
keyboard-shortcuts
capacitor
cjk
confluence
codeblocks
rack
conda
bcp
cobol
access-control
ionic3
mern
resize
32-bit
spark-structured-streaming
libxml2
emulation
undefined-behavior
swiftui
elementor
rbac
google-search
silverstripe
internet-explorer-6
r-plotly
predict
uipangesturerecognizer
system-calls
jquery-validate
httpd.conf
acumatica
kotlin-coroutines
casperjs
jqplot
synchronization
spring-data-neo4j
facebook-javascript-sdk
windows-phone-8.1
azure-table-storage
infinite-scroll
alarmmanager
casting
jbutton
mootools
osx-snow-leopard
azure-resource-manager
constants
7zip
escaping
pyside2
checkout
syslog
pythonanywhere
dockerfile
clob
qtquick2
solid-principles
webrtc
geo
data-modeling
master-pages
amazon-vpc
listviewitem
metaclass
birt
types
crossfilter
openid-connect
cucumb

mapreduce
iterable
profiler
relative-path
tensorflow-serving
std
gps
plotly
spring-cloud-dataflow
ibm-cloud-infrastructure
worksheet-function
range
xna-4.0
paypal-sandbox
moodle
.net-2.0
uipickerview
return
omniauth
pygtk
converters
liquid
stream
linear-programming
elasticsearch-5
android-cursor
mouse
gem
webdav
scheme
jscript
in-app-billing
cloud
dynamodb-queries
octobercms
node.js
recursion
minitest
asmx
perl-module
sdl
reverse-proxy
bison
el
infragistics
webhooks
model-binding
models
image-recognition
rfid
glm-math
ssas
ui-automation
service
terraform-provider-aws
dompdf
overloading
cvs
toolbar
paypal-rest-sdk
ruby-on-rails-3
paypal-ipn
c#-3.0
capybara
compare
macos
react-functional-component
tkinter-canvas
ansible-inventory
vue-router
jquery-ui-datepicker
spinner
nsmutablearray
symlink
amcharts
memory-leaks
gtk
websphere-liberty
pgadmin-4
hdfs
sqldatatypes
postback
chromium-embedded
h.264
fold
nsfilemanager
dynamic-memory-allocation
xts
twilio
meta-tags
image-segmentation
cpan
avau

Connect Error!
Sleep 50 seconds!
core-bluetooth
flask-sqlalchemy
naming-conventions
spring-test
destructuring
hugo
metrics
nat
bitcoin
android-architecture-navigation
vuetify.js
flutter-animation
xpath
hazelcast
google-ads-api
ejs
patch
linechart
mulesoft
sum
aws-sdk
callstack
dereference
transform
coronasdk
wif
folium
memory-address
datepicker
pdb
nstableview
sitecore6
glibc
abap
prolog
wsl-2
sample
smalltalk
jira-rest-api
graphviz
swagger
ibatis
laravel-query-builder
mac-address
datatables
openstreetmap
opengl
bukkit
kill
persistence
xmpp
akka-http
many-to-many
pdf.js
amazon-cognito
woocommerce
magento
processing
xamarin.ios
explorer
hangfire
expression-trees
domdocument
pug
discord.py-rewrite
android-mediacodec
unmanaged
infinite
clojure
future
saml-2.0
enumeration
jetbrains-ide
nib
compact-framework
coordinates
angular2-routing
angular-reactive-forms
spring-websocket
resque
php-7
scons
geckodriver
asp.net-3.5
django-testing
web-worker
google-chrome-devtools
c3.js
flot
msdeploy
jbos

Connect Error!
Sleep 50 seconds!
connection
autowired
http-headers
ravendb
show-hide
amazon-emr
data-binding
dynamic-linking
integer
core-animation
uinavigationcontroller
move-semantics
subquery
viewport
centos7
webcam
android-imageview
http2
named-entity-recognition
iframe
file-handling
preg-match-all
electron-builder
argparse
xslt
offset
android-pendingintent
tumblr
android-resources
google-cloud-sql
datagrid
stored-procedures
countdowntimer
airflow
memory
enterprise
jekyll
jsdoc
org-mode
mongodb-java
nan
voice
ubuntu-16.04
sorting
updates
cross-compiling
utf-16
data.table
procedure
junit
jailbreak
shared
processbuilder
devexpress
sugarcrm
autofac
media
reverse
load-balancing
spell-checking
jpeg
javafx-2
orbeon
flatten
jsch
zend-db
try-catch
p5.js
add-in
appdelegate
protocols
stringbuilder
zoo
header-files
paypal
snapshot
cross-browser
data-analysis
higher-order-functions
tomcat
flutter-web
xpages
credentials
android-testing
dataweave
gridbaglayout
facebook-fql
vue-cli
identityserver

myisam
struts2
yarnpkg
mypy
mysqldump
observable
shapefile
elements
subdirectory
exec
restful-architecture
uistoryboard
firebase-realtime-database
multiprocessing
web2py
rust-cargo
tdd
signals-slots
android-gallery
mailto
epplus
semantics
spring-mvc
clock
netflix-zuul
xcode6
router
window-functions
heap-memory
python-2.x
radio-group
datacontext
httprequest
send
shell
git-flow
styling
asterisk
populate
datetimepicker
lapply
cocoapods
cytoscape.js
assign
vpn
blockchain
python-unicode
hashtable
antlr
diagram
ios6
spring-rest
versioning
biginteger
python-3.4
exit
precision
clojurescript
visual-studio-2015
html-email
oracle-apex
dbi
restsharp
ghci
filenames
h2o
uikit
alert
segue
vi
software-design
proguard
lotus-domino
polygon
match
kafka-producer-api
onesignal
android-volley
apollo
temp-tables
marklogic
screen-scraping
avx
permalinks
vmware
visual-studio-code
oracle
composer-php
concatenation
webpage
swift4
c++builder
jsonb
google-api-java-client
fedora
javadoc
filtering
flask-restful
tale

outer-join
angularjs
xmlreader
case
string-literals
carousel
amazon-route53
android-lifecycle
asp.net-mvc-5
docker-swarm
nspredicate
quotes
sumifs
presto
literals
jasmine
itemscontrol
leaflet
child-process
xamarin.forms
interactive
innodb
tampermonkey
cmake
android-menu
slider
laravel-5.6
javascript-events
canvas
android-softkeyboard
slack-api
django-templates
self
sanitization
member
jogl
coldfusion-10
spock
prompt
rpm
pywin32
bootstrap-modal
altair
keras
imagebutton
unordered-map
webapi
collision-detection
tostring
awk
multiline
android-bitmap
submit
jwt
apache-httpclient-4.x
sql-update
mediastore
database-backups
keypress
python-3.3
twitter-oauth
cxf
properties-file
array-formulas
koa
oauth-2.0
carrierwave
impala
jakarta-mail
fadeout
spring-ws
jpql
dropdown
bson
e-commerce
teamcity
android-tablayout
okhttp
filepath
angular-material2
picasso
tree
aspect-ratio
dynamics-365
eloquent
google-sheets-api
master-detail
rendering
visual-studio-lightswitch
job-scheduling
jslint
many-to-one
re

In [66]:
print("tags_desc:",len(tags_desc))

tags_desc: 5257 visited: 5251


In [67]:
df_tags_new = pd.DataFrame()

In [68]:
df_tags_new["name"] = list(tags_clean)
df_tags_new["description"] = tags_desc

In [69]:
df_tags_new.head()

,name,description
0,database-connection,A database connection is a facility that allow...
1,automatic-ref-counting,Automatic Reference Counting (ARC) is a compil...
2,passwords,Passwords are primarily used as a way of acces...
3,quaternions,Unit quaternions are a mathematical representa...
4,packaging,Packaging is a configuration management proces...


In [83]:
df_tags_new = pd.read_csv(channel2+'/'+channel2+'_tags_desc.csv')

In [84]:
tag_stem = []
desc_stem = []
p = PorterStemmer()
for name, desc in zip(df_tags_new["name"],df_tags_new["description"]):
    t1 = ""
    output1 = ""
    t2 = ""
    output2 = ""

    for token in str(name).split('-'):
        if token.isalpha():
            #t1 += token.lower()
            output1 += p.stem(token, 0,len(token)-1)+' '
        else:
            output1 += token.lower()+' '
    tag_stem.append(output1.strip())
    
    for token in str(desc):
        if token.isalpha():
            t2 += token.lower()
        else:
            if t2:
                output2 += p.stem(t2, 0,len(t2)-1)
                t2 = ""
            output2 += token.lower()
    desc_stem.append(output2)

In [86]:
tag_stem[:10]

['.htaccess',
 '.net',
 '.net 2.0',
 '.net 3.5',
 '.net 4.0',
 '.net 4.5',
 '.net 5',
 '.net 6.0',
 '.net assembl',
 '.net core']

In [88]:
tag_stem = []
desc_stem = []
p = PorterStemmer()
for name, desc in zip(df_tags_new["name"],df_tags_new["description"]):
    tags = []
    tags_desc = []
    
    t2 = ""
    output2 = ""
    for token in str(name).split('-'):
        if token.isalpha():
            #t1 += token.lower()
            tags.append(p.stem(token, 0,len(token)-1))
        else:
            tags.append(token.lower())
    tag_stem.append(tags)
    
    for token in str(desc):
        if token.isalpha():
            t2 += token.lower()
        else:
            if t2:
                output2 += p.stem(t2, 0,len(t2)-1)
                t2 = ""
            output2 += token.lower()
    desc_stem.append(output2)

In [89]:
tag_stem[:10]

[['.htaccess'],
 ['.net'],
 ['.net', '2.0'],
 ['.net', '3.5'],
 ['.net', '4.0'],
 ['.net', '4.5'],
 ['.net', '5'],
 ['.net', '6.0'],
 ['.net', 'assembl'],
 ['.net', 'core']]

In [54]:
df_tags_new["tag_stem"] = tag_stem

In [55]:
df_tags_new["stem"] = desc_stem

In [56]:
df_tags_new.head()

,name,description,stem,tag_stem
0,.htaccess,Directory-level configuration file used by Apa...,directori-level configur file us by apach web ...,.htaccess
1,.net,Do NOT use for questions about .NET Core or .N...,do not us for question about .net core or .net...,.net
2,.net-2.0,The 2.0 version of the .NET Framework. For que...,the 2.0 version of the .net framework. for que...,.net 2.0
3,.net-3.5,"The 3.5 version of the .NET Framework, which i...","the 3.5 version of the .net framework, which i...",.net 3.5
4,.net-4.0,Version 4.0 of the .NET Framework. Use for qu...,version 4.0 of the .net framework. us for que...,.net 4.0


In [76]:
df_tags_new.to_csv(channel2+'/'+channel2+'_tags_desc.csv',index = None)

In [106]:
tag = 'automat'
start_index = df_tags_new["stem"][1].find(tag)
print("index:",start_index)
if start_index != -1:
    if ((start_index == 0) or (not df_tags_new["stem"][1][start_index-1].isalpha())) and \
        ((start_index+len(str(tag)) == len(df_tags_new["stem"][1])) or (not df_tags_new["stem"][1][start_index+len(str(tag))].isalpha())):
            
        print("find!")

index: 0
find!


In [100]:
df_tags_new["stem"][0][start_index+len(str(tag))].isalpha()

False

In [99]:
len(tag)

7

### Scikitlearn

In [59]:
scikit_tags,scikit_matched = tags_match(df_scikit_profile["answers"],tags_clean)

total tags matched: 1264


In [48]:
df_scikit_profile["tags"] = scikit_tags

In [49]:
df_scikit_profile.head()

,user_Id,cluster_Id,answer_date,answers,tags
0,54d4a1d6db8155e6700f853b,12,2015-02-06 11:14:00,I'm with family this week and so not super act...,"[import, tps, dtype, ase, el, hub, dt, om, nt,..."
1,54d4a1d6db8155e6700f853b,61,2015-02-24 22:46:00,cool\r\nok\r\n,[co]
2,54d4a1d6db8155e6700f853b,69,2015-02-25 18:59:00,yeah that sounds like a good idea\r\n,"[de, goo, ide, hat, go, ou, od]"
3,54d4a1d6db8155e6700f853b,72,2015-02-25 19:14:00,you mean == 1\r\n,"[mean, yo, ou]"
4,54d4a1d6db8155e6700f853b,75,2015-02-25 19:19:00,that is also weird ;)\r\n,"[ei, rd, hat, lso, als, ls]"


In [51]:
df_scikit_profile.to_csv(channel1+'/'+channel1+'_profile_tags.csv',index = None)

In [54]:
scikit_issue_tags,scikit_issue_matched = tags_match(df_scikit_issue["content"],tags_clean)

total tags matched: 3632


In [55]:
df_scikit_issue["tags"] = scikit_issue_tags

In [56]:
df_scikit_issue.head()

,Id,asker,time,content,lines,tags
0,1,541a528b163965c9bc2053de,2014-09-18 03:36,Hi all!\n,1,[]
1,2,544906e2db8155e6700cdd16,2014-10-23 19:59,I don't know if this is the right place but I ...,2,"[int, dvi, set, som, tr, ip, ri, om, nt, de, c..."
2,3,544906e2db8155e6700cdd16,2014-10-23 20:01,My first approach was to convert each feature ...,2,"[el, alu, rise, ri, bin, nt, val, ex, live, am..."
3,4,53a5cf04a9176b500d1ced1a,2015-01-23 15:38,This message is without content. I wonder if a...,1,"[der, ag, nt, de, ess, sage, co, yo, ent, sag,..."
4,5,541a528b163965c9bc2053de,2015-01-23 16:49,"@mac2bua, I just saw your message. It's probab...",1,"[int, el, ag, jus, sly, quest, overflow, tr, o..."


In [ ]:
df_scikit_issue.to_csv(channel1+'/'+channel1+'_issue_tags.csv',index = None)

In [60]:
len(scikit_matched)

1264

In [61]:
clusters_answered = set(df_scikit_profile["cluster_Id"])

In [62]:
len(clusters_answered)

595

In [63]:
df_scikit_issue_selected = df_scikit_issue[df_scikit_issue["Id"].isin(clusters_answered)]

In [64]:
len(df_scikit_issue_selected)

595

In [69]:
df_scikit_issue_selected.head()

,Id,asker,time,content,lines,tags
11,12,541a528b163965c9bc2053de,2015-02-06 11:14,I want to do the `check_array` empty data first\n,9,"[pty, nt, ant, ta, wan, ata, st, ch, arr, ray,..."
12,13,54d4a1d6db8155e6700f853b,2015-02-06 11:18,Most of the bug-fixes are isolated to one esti...,9,"[int, import, tps, dtype, ase, el, pipeline, i..."
13,14,54d4a1d6db8155e6700f853b,2015-02-06 11:23,it did? I didn't see that. haha I know the ope...,8,"[sis, ase, sf, el, som, ini, tr, rm, ode, ip, ..."
37,38,541a528b163965c9bc2053de,2015-02-15 16:33,Explaining the pbm in an issue is a good idea....,2,"[opl, unc, el, ini, de, bunch, goo, ch, rel, i..."
51,52,541a528b163965c9bc2053de,2015-02-24 14:06,cython path/to/file.pyx\n,2,"[cython, path, pyx, file, il, yt]"


In [81]:
issue_matched = set()
for tags in df_scikit_issue_selected["tags"]:
    issue_matched.update(tags)

In [83]:
len(issue_matched)

1741

In [85]:
print("final matched:",len(scikit_matched.union(issue_matched)))

final matched: 1977


### Angular

In [133]:
df_angular_profile.head()

,user_Id,cluster_Id,answer_date,answers
0,54e6ee3b15522ed4b3dc3c24,15,2015-03-08 18:01:00,currently only working in chrome until shadow ...
1,54e6ee3b15522ed4b3dc3c24,20,2015-03-09 02:39:00,::shadow\nhttp://www.html5rocks.com/en/tutoria...
2,54e6ee3b15522ed4b3dc3c24,100,2015-03-13 19:48:00,@mhevery that was my worry too (Dart version ...
3,54e6ee3b15522ed4b3dc3c24,113,2015-03-14 02:46:00,you donat need to inject it since itas going t...
4,54e6ee3b15522ed4b3dc3c24,118,2015-03-14 07:23:00,what version of node?\n


In [134]:
profile_stem = []
p = PorterStemmer()
for answer in df_angular_profile["answers"]:
    t2 = ""
    output2 = ""
    
    for token in str(answer):
        if token.isalpha():
            t2 += token.lower()
        else:
            if t2:
                output2 += p.stem(t2, 0,len(t2)-1)
                t2 = ""
            output2 += token.lower()
    profile_stem.append(output2)

In [135]:
df_angular_profile["answer_stem"] = profile_stem

In [136]:
df_angular_profile.head()

,user_Id,cluster_Id,answer_date,answers,answer_stem
0,54e6ee3b15522ed4b3dc3c24,15,2015-03-08 18:01:00,currently only working in chrome until shadow ...,current onli work in chrome until shadow dom e...
1,54e6ee3b15522ed4b3dc3c24,20,2015-03-09 02:39:00,::shadow\nhttp://www.html5rocks.com/en/tutoria...,::shadow\nhttp://www.html5rock.com/en/tutori/w...
2,54e6ee3b15522ed4b3dc3c24,100,2015-03-13 19:48:00,@mhevery that was my worry too (Dart version ...,@mheveri that wa my worri too (dart version o...
3,54e6ee3b15522ed4b3dc3c24,113,2015-03-14 02:46:00,you donat need to inject it since itas going t...,you donat need to inject it sinc ita go to be ...
4,54e6ee3b15522ed4b3dc3c24,118,2015-03-14 07:23:00,what version of node?\n,what version of node?\n


In [137]:
angular_tags,angular_matched = tags_match(df_angular_profile["answer_stem"],df_tags_new["name"], df_tags_new["tag_stem"])

total tags matched: 2637


In [138]:
df_angular_profile["tags"] = angular_tags

In [139]:
df_angular_profile.head()

,user_Id,cluster_Id,answer_date,answers,answer_stem,tags
0,54e6ee3b15522ed4b3dc3c24,15,2015-03-08 18:01:00,currently only working in chrome until shadow ...,current onli work in chrome until shadow dom e...,"[emulation, shadow-dom, shadow, dom, default]"
1,54e6ee3b15522ed4b3dc3c24,20,2015-03-09 02:39:00,::shadow\nhttp://www.html5rocks.com/en/tutoria...,::shadow\nhttp://www.html5rock.com/en/tutori/w...,"[http, children, shadow, html, com, styles, st..."
2,54e6ee3b15522ed4b3dc3c24,100,2015-03-13 19:48:00,@mhevery that was my worry too (Dart version ...,@mheveri that wa my worri too (dart version o...,"[foreach, dart, push, refactoring, iteration, ..."
3,54e6ee3b15522ed4b3dc3c24,113,2015-03-14 02:46:00,you donat need to inject it since itas going t...,you donat need to inject it sinc ita go to be ...,"[attachment, binding, state, set, class, selec..."
4,54e6ee3b15522ed4b3dc3c24,118,2015-03-14 07:23:00,what version of node?\n,what version of node?\n,"[version, nodes, versioning]"


In [140]:
df_angular_profile.to_csv(channel2+'/'+channel2+'_profile_tags.csv',index = None)

In [141]:
#save angular_matched
with open(channel2+'/'+channel2+'_angular_matched.txt', 'w') as fp:
    for name in angular_matched:
        # write each item on a new line
        fp.write("%s\n" % name)
    print('Done')

Done


In [77]:
df_angular_issue.head()

,Id,asker,time,content,lines,tags
0,1,54c89b1cdb8155e6700f2ac3,2015-03-07 21:41,Awesome!\n,1,[]
1,2,54e6ee3b15522ed4b3dc3c24,2015-03-07 21:53,"sweet, now we can ask angular2 questions in re...",1,"[io, time, angular, real-time]"
2,3,530cd1eb5e986b0712efb4d2,2015-03-07 22:04,Yes!\n,1,[]
3,4,54e6ee3b15522ed4b3dc3c24,2015-03-07 22:06,we're missing the [View Source] [Improve this ...,1,"[io, button, angular]"
4,5,530cd1eb5e986b0712efb4d2,2015-03-07 23:18,Good idea can you file an issue with repo?\n,1,"[ide, file]"


In [ ]:
issue_stem = []
p = PorterStemmer()
for issue in df_angular_issue["content"]:
    t2 = ""
    output2 = ""
    
    for token in str(issue):
        if token.isalpha():
            t2 += token.lower()
        else:
            if t2:
                output2 += p.stem(t2, 0,len(t2)-1)
                t2 = ""
            output2 += token.lower()
    issue_stem.append(output2)

In [ ]:
df_angular_issue["content_stem"] = issue_stem

In [127]:
angular_issue_tags,angular_issue_matched = tags_match(df_angular_issue["content_stem"],df_tags_new["name"], df_tags_new["tag_stem"])

total tags matched: 3621


In [128]:
df_angular_issue["tags"] = angular_issue_tags

In [129]:
df_angular_issue.head()

,Id,asker,time,content,lines,tags,content_stem
0,1,54c89b1cdb8155e6700f2ac3,2015-03-07 21:41,Awesome!\n,1,[],awesom!\n
1,2,54e6ee3b15522ed4b3dc3c24,2015-03-07 21:53,"sweet, now we can ask angular2 questions in re...",1,"[time, timing, real-time, angular]","sweet, now we can ask angular2 question in rea..."
2,3,530cd1eb5e986b0712efb4d2,2015-03-07 22:04,Yes!\n,1,[],ye!\n
3,4,54e6ee3b15522ed4b3dc3c24,2015-03-07 22:06,we're missing the [View Source] [Improve this ...,1,"[button, io, views, angular, ios, view]",we're miss the [view sourc] [improv thi doc] b...
4,5,530cd1eb5e986b0712efb4d2,2015-03-07 23:18,Good idea can you file an issue with repo?\n,1,[file],good idea can you file an issu with repo?\n


In [132]:
df_angular_issue.to_csv(channel2+'/'+channel2+'_issue_tags.csv',index = None)

In [32]:
clusters_answered = set(df_angular_profile["cluster_Id"])

In [33]:
len(clusters_answered)

122791

In [34]:
df_angular_issue_selected = df_angular_issue[df_angular_issue["Id"].isin(clusters_answered)]

In [35]:
len(df_angular_issue_selected)

122791

In [36]:
issue_matched = set()
for tags in df_angular_issue_selected["tags"]:
    issue_matched.update(tags)

In [37]:
print("final matched:",len(angular_matched.union(issue_matched)))

final matched: 9790


### ISPY - Tagging

In [16]:
df_tags = pd.read_csv(channel2+'/'+channel2+'_tags_desc.csv')

In [17]:
df_tags.head()

,name,description,stem,tag_stem
0,.htaccess,Directory-level configuration file used by Apa...,directori-level configur file us by apach web ...,['.htaccess']
1,.net,Do NOT use for questions about .NET Core or .N...,do not us for question about .net core or .net...,['.net']
2,.net-2.0,The 2.0 version of the .NET Framework. For que...,the 2.0 version of the .net framework. for que...,"['.net', '2.0']"
3,.net-3.5,"The 3.5 version of the .NET Framework, which i...","the 3.5 version of the .net framework, which i...","['.net', '3.5']"
4,.net-4.0,Version 4.0 of the .NET Framework. Use for qu...,version 4.0 of the .net framework. us for que...,"['.net', '4.0']"


In [18]:
df_ispy = pd.read_csv(channel2+'/'+channel2+'_ispy.csv')

In [19]:
df_ispy.head()

,issue,solution
0,[<-CODE->],I keep an eye on this channel from time to tim...
1,laco0416: [<-ISSUE->],ericmartinezr: check out my hacked together cl...
2,there isn't a.just()in Rx5 [<-LINK->] note th...,Check out this [<-LINK->]. This is the documen...
3,Shouldn'tDynamicComponentLoader#loadIntoLocati...,gionkunz: so how could i add it inside the div...
4,Will there be a material stuff for angular 2?,Currently I am using [<-LINK->] . I wonder if ...


In [22]:
ispy_issue_stem = []
ispy_solution_stem = []
p = PorterStemmer()
for answer in df_ispy["issue"]:
    t2 = ""
    output2 = ""
    
    for token in str(answer):
        if token.isalpha():
            t2 += token.lower()
        else:
            if t2:
                output2 += p.stem(t2, 0,len(t2)-1)
                t2 = ""
            output2 += token.lower()
    ispy_issue_stem.append(output2)

for answer in df_ispy["solution"]:
    t2 = ""
    output2 = ""
    
    for token in str(answer):
        if token.isalpha():
            t2 += token.lower()
        else:
            if t2:
                output2 += p.stem(t2, 0,len(t2)-1)
                t2 = ""
            output2 += token.lower()
    ispy_solution_stem.append(output2)

In [23]:
print(len(ispy_issue_stem),len(ispy_solution_stem))

6662 6662


In [26]:
df_ispy["issue_stem"] = ispy_issue_stem
df_ispy["solution_stem"] = ispy_solution_stem

In [27]:
df_ispy.head()

,issue,solution,issue_stem,solution_stem
0,[<-CODE->],I keep an eye on this channel from time to tim...,[<-code->],"i keep an ey on thi channel from time to time,..."
1,laco0416: [<-ISSUE->],ericmartinezr: check out my hacked together cl...,laco0416: [<-issu->],ericmartinezr: check out my hack togeth class ...
2,there isn't a.just()in Rx5 [<-LINK->] note th...,Check out this [<-LINK->]. This is the documen...,there isn't a.just()in rx5 [<-link->] note th...,check out thi [<-link->]. thi is the document ...
3,Shouldn'tDynamicComponentLoader#loadIntoLocati...,gionkunz: so how could i add it inside the div...,shouldn'tdynamiccomponentload#loadintolocation...,gionkunz: so how could i add it insid the div?...
4,Will there be a material stuff for angular 2?,Currently I am using [<-LINK->] . I wonder if ...,will there be a materi stuff for angular 2?,current i am us [<-link->] . i wonder if i am ...


In [28]:
df_ispy.to_csv(channel2+'/'+channel2+'_ispy.csv')

In [92]:
ispy_tags,ispy_matched = tags_match(df_ispy["issue_stem"],df_tags_new["name"], tag_stem)

total tags matched: 1284


In [93]:
ispy_s_tags,ispy_s_matched = tags_match(df_ispy["solution_stem"],df_tags_new["name"], tag_stem)

total tags matched: 1510


In [94]:
df_ispy["issue_tags"] = ispy_tags
df_ispy["solution_tags"] = ispy_s_tags

In [95]:
df_ispy.head()

,issue,solution,issue_stem,solution_stem,issue_tags,solution_tags
0,[<-CODE->],I keep an eye on this channel from time to tim...,[<-code->],"i keep an ey on thi channel from time to time,...",[],"[channel, chat, one-to-one, time, timing]"
1,laco0416: [<-ISSUE->],ericmartinezr: check out my hacked together cl...,laco0416: [<-issu->],ericmartinezr: check out my hack togeth class ...,[],"[base, class, config, routes, routing]"
2,there isn't a.just()in Rx5 [<-LINK->] note th...,Check out this [<-LINK->]. This is the documen...,there isn't a.just()in rx5 [<-link->] note th...,check out thi [<-link->]. thi is the document ...,"[difference, observable, reactive, rxjs]","[document, documentation, paste, replace, rxjs..."
3,Shouldn'tDynamicComponentLoader#loadIntoLocati...,gionkunz: so how could i add it inside the div...,shouldn'tdynamiccomponentload#loadintolocation...,gionkunz: so how could i add it insid the div?...,[add],"[add, angular, angular-components, components,..."
4,Will there be a material stuff for angular 2?,Currently I am using [<-LINK->] . I wonder if ...,will there be a materi stuff for angular 2?,current i am us [<-link->] . i wonder if i am ...,"[angular, angular-material, materialize]","[go, path, using]"


In [96]:
df_ispy.to_csv(channel2+'/'+channel2+'_ispy.csv')

## Similarity Analysis - User Profile

In [119]:
df_name = pd.read_json("../Data/Gitter_Channels/"+channel2+"/"+channel2+'_user.json')

In [120]:
df_name.head()

,id,username,displayName
0,529c69aded5ab0b3bf04d139,abraham,Abraham Williams
1,529c69b1ed5ab0b3bf04d145,simonrenoult,Simon Renoult
2,529c69b3ed5ab0b3bf04d14a,leemour,Viacheslav Ptsarev
3,529c69b7ed5ab0b3bf04d157,paullaros,Paul Laros
4,529c69b8ed5ab0b3bf04d158,kars7e,Karol Stępniewski


In [121]:
df_gitter = pd.read_csv(channel2+'/'+channel2+'_profile_tags.csv')

In [122]:
df_gitter.head()

,user_Id,cluster_Id,answer_date,answers,answer_stem,tags
0,54e6ee3b15522ed4b3dc3c24,15,2015-03-08 18:01:00,currently only working in chrome until shadow ...,current onli work in chrome until shadow dom e...,"['emulation', 'shadow-dom', 'shadow', 'dom', '..."
1,54e6ee3b15522ed4b3dc3c24,20,2015-03-09 02:39:00,::shadow\nhttp://www.html5rocks.com/en/tutoria...,::shadow\nhttp://www.html5rock.com/en/tutori/w...,"['http', 'children', 'shadow', 'html', 'com', ..."
2,54e6ee3b15522ed4b3dc3c24,100,2015-03-13 19:48:00,@mhevery that was my worry too (Dart version ...,@mheveri that wa my worri too (dart version o...,"['foreach', 'dart', 'push', 'refactoring', 'it..."
3,54e6ee3b15522ed4b3dc3c24,113,2015-03-14 02:46:00,you donat need to inject it since itas going t...,you donat need to inject it sinc ita go to be ...,"['attachment', 'binding', 'state', 'set', 'cla..."
4,54e6ee3b15522ed4b3dc3c24,118,2015-03-14 07:23:00,what version of node?\n,what version of node?\n,"['version', 'nodes', 'versioning']"


In [123]:
df_issue = pd.read_csv(channel2+'/'+channel2+'_issue_tags.csv')

In [124]:
df_issue.head()

,Id,asker,time,content,lines,tags,content_stem
0,1,54c89b1cdb8155e6700f2ac3,2015-03-07 21:41,Awesome!\n,1,[],awesom!\n
1,2,54e6ee3b15522ed4b3dc3c24,2015-03-07 21:53,"sweet, now we can ask angular2 questions in re...",1,"['time', 'timing', 'real-time', 'angular']","sweet, now we can ask angular2 question in rea..."
2,3,530cd1eb5e986b0712efb4d2,2015-03-07 22:04,Yes!\n,1,[],ye!\n
3,4,54e6ee3b15522ed4b3dc3c24,2015-03-07 22:06,we're missing the [View Source] [Improve this ...,1,"['button', 'io', 'views', 'angular', 'ios', 'v...",we're miss the [view sourc] [improv thi doc] b...
4,5,530cd1eb5e986b0712efb4d2,2015-03-07 23:18,Good idea can you file an issue with repo?\n,1,['file'],good idea can you file an issu with repo?\n


In [125]:
df_github = pd.read_csv(channel2+'/'+channel2+'_user_meta_tags.csv')

In [126]:
df_github.head()

,User,Repo_num,Repo_languages,Repo_topics,user_Id
0,7444,3,"['Ruby', 'Shell', 'HTML']",[],5f727524d73408ce4ff021fe
1,01CodeLT,3,"['TypeScript', 'HTML', 'SCSS', 'JavaScript', '...","['meeting', 'jw', 'bible', 'epub-reader', 'jeh...",5f86b560d73408ce4ff17d63
2,02arpit07,50,"['TypeScript', 'Julia', 'Go', 'PHP', 'SCSS', '...",[],5f2f0157d73408ce4febb81b
3,03williammoses,4,"['TeX', 'Java', 'Roff']",[],5e5c9d01d73408ce4fdb4239
4,04raghuramvugs,8,"['CSS', 'TypeScript', 'JavaScript', 'HTML']",[],5bbf743bd73408ce4fab237b


In [127]:
print(len(df_github))

20114


In [87]:
df_angular_sorted.head(10)

,user_Id,count
0,57787d7fc2f0db084a2127f1,11700
1,5666e77c16b6c7089cbde7d5,7588
2,5724bc6d659847a7aff549ce,6538
3,576a94bdc2f0db084a1f5332,4644
4,5515c77715522ed4b3ddf13e,4381
5,55b409ef0fc9f982beaafb85,3974
6,54a8b22bdb8155e6700e57c1,3872
7,563a5bc416b6c7089cb97cfa,3033
8,579228bd40f3a6eec05c03aa,2521
9,54d28ce0db8155e6700f7486,2219


In [88]:
df_angular_sorted["user_Id"][0:10].values

array(['57787d7fc2f0db084a2127f1', '5666e77c16b6c7089cbde7d5',
       '5724bc6d659847a7aff549ce', '576a94bdc2f0db084a1f5332',
       '5515c77715522ed4b3ddf13e', '55b409ef0fc9f982beaafb85',
       '54a8b22bdb8155e6700e57c1', '563a5bc416b6c7089cb97cfa',
       '579228bd40f3a6eec05c03aa', '54d28ce0db8155e6700f7486'],
      dtype=object)

In [160]:
all_profile = []
for count, user_Id_selected in enumerate(df_github["user_Id"]):
    user_profile = {}
    github_tags = []
    gitter_tags = []
    #solution_tags = []
    
    #if user_Id_selected not in user_profile.keys():
    #print(user_Id_selected)
    user_profile["user_Id"] = user_Id_selected
    user_profile["name"] = df_github[df_github["user_Id"] == user_Id_selected]["User"].values[0]
    
    github_tags.extend(df_github[df_github["user_Id"] == user_Id_selected]["Repo_languages"].apply(eval)[count])
    github_tags.extend(df_github[df_github["user_Id"] == user_Id_selected]["Repo_topics"].apply(eval)[count])
    
    '''
    for cluster in list(df_gitter[df_gitter["user_Id"] == user_Id_selected]["cluster_Id"].values):
        gitter_tags.extend(df_issue[df_issue["Id"] == cluster]["tags"].apply(eval)[cluster-1])
   
    for tags in df_gitter[df_gitter["user_Id"] == "54e6ee3b15522ed4b3dc3c24"]["tags"].apply(eval):
        gitter_tags.extend(tags)
    '''
    
    try:
        user_profile["user_Id"] = user_Id_selected
        user_profile["name"] = df_github[df_github["user_Id"] == user_Id_selected]["User"].values[0]
    
        github_tags.extend(df_github[df_github["user_Id"] == user_Id_selected]["Repo_languages"].apply(eval)[count])
        github_tags.extend(df_github[df_github["user_Id"] == user_Id_selected]["Repo_topics"].apply(eval)[count])
    
        for cluster in list(df_gitter[df_gitter["user_Id"] == user_Id_selected]["cluster_Id"].values):
            gitter_tags.extend(df_issue[df_issue["Id"] == cluster]["tags"].apply(eval)[cluster-1])
   
        for tags in df_gitter[df_gitter["user_Id"] == "54e6ee3b15522ed4b3dc3c24"]["tags"].apply(eval):
            gitter_tags.extend(tags)
    except Exception as e:
        print(count, user_Id_selected,":",e)
    
    user_profile["GitHub_Tags"] = github_tags
    user_profile["Gitter_Tags"] = gitter_tags
    
    all_profile.append(user_profile)
    #issue_tags.extend(solution_tags)
    #print(set(github_tags))
    #print(set(gitter_tags))
    #print(solution_tags)

2009 570fc260187bb6f0eadf743e : name 'nan' is not defined
2022 55dec8030fc9f982beaec021 : name 'nan' is not defined
4715 550550a815522ed4b3dd4847 : name 'nan' is not defined
4910 578f48e9c2f0db084a23d2b6 : name 'nan' is not defined
9056 57bbd68740f3a6eec060d6f9 : name 'nan' is not defined
9271 5724bc6d659847a7aff549ce : name 'nan' is not defined
11473 56da5a52e610378809c518bc : name 'nan' is not defined
12634 569555cb16b6c7089cc23cb5 : name 'nan' is not defined
13881 54d28ce0db8155e6700f7486 : name 'nan' is not defined
16461 57787d7fc2f0db084a2127f1 : name 'nan' is not defined
18700 5666e77c16b6c7089cbde7d5 : name 'nan' is not defined
18744 55b409ef0fc9f982beaafb85 : name 'nan' is not defined


In [131]:
len(all_profile)

20114

In [167]:
import json
with open(channel2+'/'+channel2+'user_alltags.json', 'w') as outfile:
    json.dump(all_profile, outfile)

In [168]:
df = pd.read_json(channel2+'/'+channel2+'user_alltags.json')

In [169]:
df.head()

,user_Id,name,GitHub_Tags,Gitter_Tags
0,5f727524d73408ce4ff021fe,7444,"[Ruby, Shell, HTML, Ruby, Shell, HTML]","[emulation, shadow-dom, shadow, dom, default, ..."
1,5f86b560d73408ce4ff17d63,01CodeLT,"[TypeScript, HTML, SCSS, JavaScript, Python, m...","[emulation, shadow-dom, shadow, dom, default, ..."
2,5f2f0157d73408ce4febb81b,02arpit07,"[TypeScript, Julia, Go, PHP, SCSS, JavaScript,...","[emulation, shadow-dom, shadow, dom, default, ..."
3,5e5c9d01d73408ce4fdb4239,03williammoses,"[TeX, Java, Roff, TeX, Java, Roff]","[angular, cpanel, angular-universal, emulation..."
4,5bbf743bd73408ce4fab237b,04raghuramvugs,"[CSS, TypeScript, JavaScript, HTML, CSS, TypeS...","[http, commit, github, updates, com, updating,..."


In [170]:
df.to_csv(channel2+'/'+channel2+'_alltags.csv',index = None)

In [139]:
df_github[df_github["user_Id"] == "570fc260187bb6f0eadf743e"]["Repo_topics"].apply(eval)[2009]

['object-oriented-programming',
 'design-patterns',
 'phpunit',
 'php-design-pattern']

In [137]:
df_gitter[df_gitter["user_Id"] == "570fc260187bb6f0eadf743e"]["cluster_Id"].values

array([135496, 136041, 136046, 136051, 136100, 137047, 137055, 137135,
       137998, 138067, 138117, 138119, 138678, 139648, 140124, 140170,
       140525, 140534, 140568, 141000, 141001, 141138, 142347, 142357,
       142358, 142359, 142360, 142362, 142364, 142535, 142546, 143450,
       143545, 143914, 144043, 144426, 144600, 146708, 146778, 146826,
       146829, 147335, 147340, 147341, 147393, 147395, 150165, 150170,
       151777, 158979, 160094, 160097, 160326, 160327, 160332, 161646,
       180266, 180274, 182690, 182754, 182762, 182775, 186237, 190760,
       192324, 192327, 194997, 195073, 195075, 195101, 205218, 205265,
       205872, 208607, 208639, 208643, 208649, 209133, 209268, 210035,
       210085, 210090, 210096, 210100, 210103, 210104, 210105, 211914,
       211971, 213409, 214065, 216501, 216594, 216862, 261349, 288722,
       289075, 289799, 412031], dtype=int64)

In [143]:
for cluster in list(df_gitter[df_gitter["user_Id"] == "570fc260187bb6f0eadf743e"]["cluster_Id"].values):
    print(cluster,df_issue[df_issue["Id"] == cluster]["tags"].apply(eval)[cluster-1])

135496 ['components', 'load', 'cascade', 'file', 'javascript', 'loading', 'dropdown', 'input']
136041 ['attachment', 'dos', 'rxjs', 'lines', 'require', 'handler', 'using', 'file', 'line', 'wait', 'events']
136046 ['attachment', 'documentation', 'document']
136051 ['dos', 'directive', 'handler', 'attributes', 'exists', 'indexing', 'one-to-one', 'put', 'dom', 'html', 'element', 'elements', 'handle', 'apply']
136100 []
137047 ['http', 'app-config', 'system', 'http-get', 'host', 'base', 'refresh', 'location', 'views', 'routing', 'params', 'routes', 'angular2-routing', 'url-routing', 'get', 'indexing', 'load', 'file', 'config', 'static', 'html', 'styles', 'styling', 'loading', 'angular', 'angular-routing', 'hosting', 'ads', 'windows', 'window', 'coding-style', 'url', 'fixed', 'src', 'view', 'https']
137055 ['extend', 'stack-trace', 'extends', 'stack', 'trace']
137135 ['localization', 'variables', 'locale', 'time', 'generics', 'timing', 'initialization', 'local-variables', 'generator', 'for-

NameError: name 'nan' is not defined

In [159]:
df_issue[df_issue["Id"] == 140534]["tags"]

140533    ['naming', 'add', 'variables', 'forms', 'set',...
Name: tags, dtype: object

In [153]:
df_issue[df_issue["Id"] == 140525]["tags"].apply(eval)[140524]

['associations',
 'lines',
 'accessibility',
 'plugins',
 'arrays',
 'undefined',
 'function',
 'rewrite',
 'associative-array',
 'normalization',
 'using',
 'live',
 'dom',
 'line',
 'angular']

## Similar Issue by Tag

In [171]:
df_issue = pd.read_csv(channel2+'/'+channel2+'_issue_tags.csv')

In [173]:
df_issue.head()

,Id,asker,time,content,lines,tags,content_stem
0,1,54c89b1cdb8155e6700f2ac3,2015-03-07 21:41,Awesome!\n,1,[],awesom!\n
1,2,54e6ee3b15522ed4b3dc3c24,2015-03-07 21:53,"sweet, now we can ask angular2 questions in re...",1,"['time', 'timing', 'real-time', 'angular']","sweet, now we can ask angular2 question in rea..."
2,3,530cd1eb5e986b0712efb4d2,2015-03-07 22:04,Yes!\n,1,[],ye!\n
3,4,54e6ee3b15522ed4b3dc3c24,2015-03-07 22:06,we're missing the [View Source] [Improve this ...,1,"['button', 'io', 'views', 'angular', 'ios', 'v...",we're miss the [view sourc] [improv thi doc] b...
4,5,530cd1eb5e986b0712efb4d2,2015-03-07 23:18,Good idea can you file an issue with repo?\n,1,['file'],good idea can you file an issu with repo?\n


In [175]:
import math
import re
from collections import Counter

WORD = re.compile(r"\w+")


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)


text1 = "This is a foo bar sentence ."
text2 = "This sentence is similar to a foo bar sentence ."

vector1 = text_to_vector(text1)
vector2 = text_to_vector(text2)

cosine = get_cosine(vector1, vector2)
print(vector1)
print(vector2)
print("Cosine:", cosine)

Counter({'This': 1, 'is': 1, 'a': 1, 'foo': 1, 'bar': 1, 'sentence': 1})
Counter({'sentence': 2, 'This': 1, 'is': 1, 'similar': 1, 'to': 1, 'a': 1, 'foo': 1, 'bar': 1})
Cosine: 0.8616404368553293
